In [2]:
import json
import numpy as np

# Load the data
data_path = './data/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

gpt_anno_path = './data/claude_check.json'
with open(gpt_anno_path, 'r') as f:
    gpt_anno = json.load(f)
gpt_change_path = './data/claude_change.json' 
with open(gpt_change_path, 'r') as f:
    gpt_change = json.load(f)  

correct = 0
total = 0

for i, d in enumerate(data):
    ref_t1 = d['ref_t1_score']
    ref_t2 = d['ref_t2_score']
    confidence = d['confidence']
    gt_labels = int(gpt_anno[i])
    change = gpt_change[i]
    if change == 1:
        gt_labels = 1 - gt_labels

    if ref_t1 > ref_t2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1
    
    total += 1
    if d['rc_labels'] == gt_labels:
        print(i,gt_labels)
        correct += 1

# Calculate accuracy for the current threshold
accuracy = correct / total if total > 0 else 0



print(accuracy)


0 0
2 1
3 1
5 0
6 1
8 0
9 0
11 0
12 1
13 0
14 0
16 1
19 0
20 0
21 1
22 0
23 1
24 1
25 1
28 0
30 0
31 0
33 1
34 1
36 0
38 0
40 0
41 1
43 0
44 0
46 0
48 0
50 0
51 1
52 0
54 0
55 1
56 1
57 0
59 0
63 1
64 1
65 1
66 0
67 1
68 0
71 1
72 0
73 1
74 1
75 1
78 0
79 0
82 1
83 0
85 0
86 1
87 1
88 1
90 0
91 0
92 1
93 0
95 0
97 1
98 1
100 0
102 1
104 0
107 0
109 1
111 1
112 1
113 0
114 0
115 1
116 1
117 0
120 1
122 0
123 0
124 1
125 0
126 0
127 1
129 0
132 1
134 1
135 0
136 1
137 0
138 0
140 0
141 0
142 1
143 0
144 0
145 0
148 1
149 0
150 1
151 1
153 0
154 0
155 0
156 1
158 0
159 0
162 1
163 1
165 1
166 1
168 0
169 0
170 1
171 0
172 1
173 1
175 0
176 0
178 1
179 1
180 0
184 1
187 0
188 1
189 0
190 1
191 1
192 0
194 0
195 0
198 0
199 1
200 0
202 0
204 1
206 1
208 0
210 1
211 1
216 1
217 0
218 0
219 0
221 0
224 0
225 0
226 0
228 1
232 0
237 1
238 1
239 1
240 1
241 1
242 0
244 1
245 0
246 1
247 1
248 1
250 1
252 1
256 0
258 0
259 1
263 1
265 0
266 1
268 1
270 0
272 1
273 1
277 0
278 1
280 0
281 0
283 1

IndexError: list index out of range

BoW result

In [18]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data
data_path = './data/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Initialize vectorizer
vectorizer = CountVectorizer()

# Initialize variables
threshold = 1e-06
correct = 0
total = 0

# Process each item in data
for i,d in enumerate(data):
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    confidences = d['confidence']
    gt_labels = int(gpt_anno[i])
    change = gpt_change[i]
    if change == 1:
         gt_labels = 1 - int(gt_labels)

    # Vectorize the texts
    vectors = vectorizer.fit_transform([ref, text1, text2])
    
    # Compute cosine similarity
    cosine_sim_ref_text1 = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    cosine_sim_ref_text2 = cosine_similarity(vectors[0:1], vectors[2:3])[0][0]

    # Determine the label based on cosine similarity and threshold
    if cosine_sim_ref_text1 > cosine_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)


Accuracy: 0.7676240208877284


TF-IDF result

In [20]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data
data_path = './data/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Initialize vectorizer
vectorizer = TfidfVectorizer()

# Initialize variables
threshold = 1e-06
correct = 0
total = 0

# Process each item in data
for i,d in enumerate(data):
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    confidences = d['confidence']
    gt_labels = int(gpt_anno[i])
    change = gpt_change[i]
    if change == 1:
        gt_labels = 1 - gt_labels

    # Skip if confidence is too low
    if confidences < 0:
        continue

    # Vectorize the texts
    vectors = vectorizer.fit_transform([ref, text1, text2])
    
    # Compute cosine similarity
    cosine_sim_ref_text1 = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    cosine_sim_ref_text2 = cosine_similarity(vectors[0:1], vectors[2:3])[0][0]

    # Determine the label based on cosine similarity and threshold
    if abs(cosine_sim_ref_text1 - cosine_sim_ref_text2) < threshold:
        d['rc_labels'] = 2
    elif cosine_sim_ref_text1 > cosine_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)
print('Total:', total)


Accuracy: 0.7519582245430809
Total: 383


In [21]:
import json
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load data
data_path = './data/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Load SimCSE model and tokenizer
model_name = "princeton-nlp/sup-simcse-roberta-large"  # Use appropriate SimCSE model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

# Initialize variables
threshold = 1e-06
correct = 0
total = 0

# Process each item in data
for i,d in enumerate(data):
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    confidences = d['confidence']
    gt_labels = int(gpt_anno[i])
    change = gpt_change[i]
    if change == 1:
        gt_labels = 1 - int(gt_labels)

    # Skip if confidence is too low
    if confidences < 0:
        continue

    # Compute embeddings
    embedding_ref = get_embedding(ref)
    embedding_text1 = get_embedding(text1)
    embedding_text2 = get_embedding(text2)

    # Compute cosine similarity
    cosine_sim_ref_text1 = cosine_similarity(embedding_ref, embedding_text1)[0][0]
    cosine_sim_ref_text2 = cosine_similarity(embedding_ref, embedding_text2)[0][0]

    # Determine the label based on cosine similarity and threshold
    if abs(cosine_sim_ref_text1 - cosine_sim_ref_text2) < threshold:
        d['rc_labels'] = 2
    elif cosine_sim_ref_text1 > cosine_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)


/opt/conda/envs/refchecker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accuracy: 0.7467362924281984


In [5]:
import json
from bert_score import score

# Load data
data_path = './data/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Initialize variables
correct = 0
total = 0

# Process each item in data
for i, d in enumerate(data):
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    gt_labels = int(gpt_anno[i])
    change = gpt_change[i]
    
    if change == 1:
         gt_labels = 1 - int(gt_labels)

    # Calculate BERTScore
    P, R, F1 = score([text1], [ref], lang='en', verbose=True)
    bert_score_text1 = F1[0].item()  # Get F1 score for text1
    P, R, F1 = score([text2], [ref], lang='en', verbose=True)
    bert_score_text2 = F1[0].item()  # Get F1 score for text2

    # Determine the label based on BERTScore
    if bert_score_text1 > bert_score_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)
np.save('data/bertscore.npy', correct / total)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  1.65it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 39.79it/s]


...

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 21.58it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 607.61it/s]

done in 0.05 seconds, 19.41 sentences/sec
Accuracy: 0.762402088772846


In [6]:
import numpy as np
a = np.load('data/bertscore.npy', allow_pickle=True)
print(a)

0.762402088772846


Sentence-BERT Similarity



In [7]:
import json
from sentence_transformers import SentenceTransformer, util

# Load data
data_path = './data/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Initialize Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose a different model

# Initialize variables
correct = 0
total = 0

# Process each item in data
for i, d in enumerate(data):
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    gt_labels = int(gpt_anno[i])
    change = gpt_change[i]
    
    if change == 1:
        gt_labels = 1 - int(gt_labels)

    # Compute embeddings
    embeddings = model.encode([ref, text1, text2], convert_to_tensor=True)

    # Compute cosine similarity
    cosine_sim_ref_text1 = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    cosine_sim_ref_text2 = util.pytorch_cos_sim(embeddings[0], embeddings[2]).item()

    # Determine the label based on cosine similarity
    if cosine_sim_ref_text1 > cosine_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)


/opt/conda/envs/refchecker/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Accuracy: 0.7650130548302873


Jarcard Similarity (n-gram)

In [8]:
import json
from sklearn.feature_extraction.text import CountVectorizer

# Load data
data_path = './data/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Function to calculate Jaccard Similarity
def jaccard_similarity(text1, text2, n=2):
    # Create n-grams
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(n, n))
    ngrams1 = set(vectorizer.fit_transform([text1]).toarray()[0])
    ngrams2 = set(vectorizer.fit_transform([text2]).toarray()[0])
    
    # Calculate intersection and union
    intersection = len(ngrams1.intersection(ngrams2))
    union = len(ngrams1.union(ngrams2))
    
    # Jaccard Similarity
    return intersection / union if union != 0 else 0

# Initialize variables
correct = 0
total = 0

# Process each item in data
for i, d in enumerate(data):
    ref = d['reference']
    text1 = d['text1']
    text2 = d['text2']
    gt_labels = int(gpt_anno[i])
    change = gpt_change[i]
    
    if change == 1:
        gt_labels = 1 - int(gt_labels)

    # Calculate Jaccard Similarity for n-grams
    jaccard_sim_ref_text1 = jaccard_similarity(ref, text1, n=2)  # Change n as needed
    jaccard_sim_ref_text2 = jaccard_similarity(ref, text2, n=2)

    # Determine the label based on Jaccard Similarity
    if jaccard_sim_ref_text1 > jaccard_sim_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)


Accuracy: 0.556135770234987


WMD

In [9]:
import json
import gensim.downloader as api
from gensim.corpora.dictionary import Dictionary

# Load data
data_path = './data/score.json'
with open(data_path, 'r') as f:
    data = json.load(f)

# Load pre-trained Word2Vec model
model = api.load("glove-wiki-gigaword-50")  # You can use other models like 'word2vec-google-news-300'

# Initialize variables
correct = 0
total = 0

# Preprocess text by tokenizing
def preprocess(text):
    return text.lower().split()

# Process each item in data
for i, d in enumerate(data):
    ref = preprocess(d['reference'])
    text1 = preprocess(d['text1'])
    text2 = preprocess(d['text2'])
    gt_labels = int(gpt_anno[i])
    change = gpt_change[i]
    
    if change == 1:
        gt_labels = 1 - int(gt_labels)

    # Compute WMD between reference and text1, text2
    wmd_ref_text1 = model.wmdistance(ref, text1)
    wmd_ref_text2 = model.wmdistance(ref, text2)

    # Determine the label based on WMD (lower distance = more similar)
    if wmd_ref_text1 < wmd_ref_text2:
        d['rc_labels'] = 0
    else:
        d['rc_labels'] = 1

    # Count correct predictions
    total += 1
    if d['rc_labels'] == gt_labels:
        correct += 1

# Print the accuracy
print('Accuracy:', correct / total)


[==================================================] 100.0% 66.0/66.0MB downloaded
Accuracy: 0.7806788511749347
